## Lecture du réseau routier

In [1]:
import geopandas as gpd
import momepy
import networkx as nx

file_reseau = "C:/Users/David/Optim/Optimisation/donnees/Troncon route Pacy et environ.shx"

# Lire le shapefile avec GeoPandas
gdf = gpd.read_file(file_reseau)
# Vérifier que la géométrie contient des LineStrings
if not all(gdf.geometry.type.isin(["LineString", "MultiLineString"])):
    raise ValueError("Le fichier doit contenir uniquement des lignes.")


# Convertir en graphe avec la méthode 'primal' (chaque intersection est un nœud)
g_reseau = momepy.gdf_to_nx(gdf, approach='primal')


c:\Users\David\Optim\Optimisation\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Lecture fichier points de livraision

In [2]:
file_livr = "C:/Users/David/Optim/Optimisation/donnees/Points livraison.shx"

# Lire le shapefile avec GeoPandas
points_livr = gpd.read_file(file_livr)

## Recherche des points du réseau les plus près des points de livraison

In [15]:
nodes_livr = points_livr.geometry.get_coordinates()
nodes_livr['type'] = points_livr['type']

In [25]:
def type_point_reseau(g_reseau,nodes_livr):

    import numpy as np
    nodes_list = list(g_reseau.nodes)
    dico_attribut_nodes = dict()

    for idx, row in nodes_livr.iterrows():
        x_liv = row['x']
        y_liv = row['y']
                
        nearest_node = nodes_list[0]
        X = nearest_node[0]
        Y = nearest_node[1]

        d_nearest = np.sqrt((x_liv-X)**2 + (y_liv-Y)**2)

        for n in nodes_list[1:]:
            X = n[0]
            Y = n[1]

            d = np.sqrt((x_liv-X)**2 + (y_liv-Y)**2)
            if d < d_nearest:
                nearest_node = n
                d_nearest = d
        
        dico_attribut_nodes[nearest_node] = {'type':row['type']}

    for n in nodes_list:
        if n not in dico_attribut_nodes.keys():
            dico_attribut_nodes[n] = {'type':'interieur'}   

        
    return dico_attribut_nodes

        

In [ ]:
dico_attribut_nodes = type_point_reseau(g_reseau,nodes_livr)
nx.set_node_attributes(g_reseau, dico_attribut_nodes)

In [29]:
nx.get_node_attributes(g_reseau,'type')

{(586492.1, 6884086.6): 'interieur',
 (584940.1, 6885800.8): 'interieur',
 (584697.4, 6886001.7): 'interieur',
 (584484.3, 6886174.1): 'interieur',
 (582063.5, 6881389.5): 'interieur',
 (582137.8, 6881233.1): 'c17',
 (584554.7, 6886422.2): 'interieur',
 (584830.8, 6886293.6): 'interieur',
 (579567.1, 6880349.6): 'interieur',
 (579458.1, 6880411.4): 'interieur',
 (579983.0, 6880393.6): 'interieur',
 (580003.4, 6880313.4): 'interieur',
 (579023.5, 6882296.4): 'interieur',
 (579236.4, 6882059.8): 'interieur',
 (578564.3, 6883680.5): 'interieur',
 (578668.3, 6882964.0): 'interieur',
 (579585.0, 6882529.1): 'interieur',
 (580386.1, 6882590.9): 'interieur',
 (580250.4, 6882612.0): 'interieur',
 (580612.3, 6882748.9): 'interieur',
 (582163.0, 6880437.1): 'interieur',
 (582488.9, 6880270.8): 'interieur',
 (581998.5, 6880952.4): 'c12',
 (581767.2, 6880784.4): 'c10',
 (582127.1, 6881022.8): 'interieur',
 (581620.8, 6880600.8): 'interieur',
 (582280.5, 6880015.6): 'interieur',
 (583016.0, 6880004